In [1]:
import os
import cv2
import h5py
import keras
import shutil
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from keras.models import Sequential, load_model, Model
from keras.layers import InputLayer, Conv2D, MaxPooling2D, UpSampling2D, Dropout, concatenate, Conv2DTranspose, Dense, Conv2D, Flatten
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
size = 256
batch_size = 100
channels = 1

dataset_path = "images/0"
dataset_test_path = "images/1"
resized_path = "squared_images/"
train_path = "bw_images/"
validate_path = "gray_images/"
test_path = "bw_test_images/"

models_path = "saved_parameters/"
saved_model = "saved_model.h5"
saved_weights = "saved_weights.h5"

In [12]:
# Create directory
def create_dir(folder_name):
    folder = os.path.join(folder_name)
    if not os.path.exists(folder):
        os.mkdir(folder)
    return folder
                          
def save_path(dir_path, file, appended_name):
    file_base = os.path.basename(file)
    file_name, file_ext = os.path.splitext(file_base)   
    return dir_path + file_name + appended_name + file_ext

# Create and save black and white images
def convert_RGB2BW(dataset_path, train_path, validate_path):
    bw_path = create_dir(train_path)
    grayscale_path = create_dir(validate_path)
    for file in os.listdir(dataset_path):
        image = Image.open(os.path.join(dataset_path, file))
        
        # Convert image to grayscale based on Lighting
        grayscale = image.convert('L')
        grayscale = np.asarray(grayscale).copy()
        gray_image = Image.fromarray(grayscale)
        gray_image.save(save_path(grayscale_path, file, "-GR"))
        
        # Turn darker pixels black (0) and lighter pixels white (255)
        grayscale[grayscale < 128] = 0
        grayscale[grayscale >= 128] = 255
        bw_image = Image.fromarray(grayscale)
        bw_image.save(save_path(bw_path, file, "-BW"))
        
# Preprocess images: creates a white image and pastes data into white image
def square_images(dataset_path, size):
    dir_path = create_dir("squared_images/")
    for file in os.listdir(dataset_path):
        image = Image.open(os.path.join(dataset_path, file))
        width, height = image.size
        if(width != height and width != size):
            squared_image = image.resize((size,size), Image.LANCZOS)
            #squared_image.paste(image)
            squared_image.save(save_path(dir_path, file, "_SQ"))
            
def load_test(dataset_test_path, test_path, size):
    dir_path = create_dir(test_path)
    for count, file in zip(range(0, size), os.listdir(dataset_test_path)):
        convert_RGB2BW

def split_test(source_path, dest_path):
    for count, file in zip(range(0,10), os.listdir(source_path)):
        shutil.move(os.path.join(source_path, file), os.path.join(dest_path, file))
                                

In [8]:
# Load training and validating images with cv2 to capture channel dim
def load_binary_gray(dir_path):
    data_arr = []
    for file in os.listdir(dir_path):
        img_read = cv2.imread(os.path.join(dir_path, file), 0)
        img_read = np.array(img_read)
        data_arr.append(img_read)
    return data_arr

def test_load(dir_path):
    data_arr = []
    counter = 0
    for file in sorted(os.listdir(dir_path)):
        img_read = cv2.imread(os.path.join(dir_path, file), 0)
        img_read = np.array(img_read)
        print(file)
        print(img_read.shape)
        data_arr.append(img_read)
        counter += 1
        if counter == 2:
            return data_arr
    return data_arr

# Calculate epoch size to traverse all training data
def step_size(dir_path, batch_size):
    counter = 0
    for file in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, file)):
            counter += 1
    return int(counter/batch_size)

In [ ]:
# Resize Image
square_images(dataset_path, size)

In [13]:
# Image Preprocessing
convert_RGB2BW(resized_path, train_path, validate_path)
split(train_path, test_path)

OSError: [Errno 24] Too many open files: 'squared_images/'

In [ ]:
# Load images into arrays
'''
train_arr = load_binary_gray(train_path)
validate_arr = load_binary_gray(validate_path)
'''
train_arr = test_load(train_path)
validate_arr = test_load(validate_path)

# Convert image array into numpy array
train_np = np.array(train_arr)
validate_np = np.array(validate_arr)

train_np = train_np.astype('float32')
validate_np = validate_np.astype('float32')

# Reshape to include channel dim for grayscale/black and white images
train_np = train_np.reshape(train_np.shape[0], size, size, 1)
validate_np = validate_np.reshape(validate_np.shape[0], size, size, 1)

# Normalize
train_np = train_np/255
validate_np = validate_np/255

In [ ]:
# Check data size, dimensions, and channels

print(train_np.shape)
print(train_np.dtype)
print(validate_np.shape)
print(validate_np.dtype)

print(train_np[0].shape)
plt.imshow(train_np[0].squeeze(), cmap='gray')


In [ ]:
# Autoencoder model
auto = Sequential()
auto.add(InputLayer(input_shape=(train_np.shape[1:])))

# Encoder - smaller filters, larger feature map
auto.add(Conv2D(16, (3,3), strides=(2,2), padding = 'same', activation='relu'))
auto.add(MaxPooling2D((2,2), padding='same'))
auto.add(Conv2D(32, (3,3), strides=(2,2), padding='same', activation='relu'))
auto.add(MaxPooling2D((2,2), padding='same'))
auto.add(Conv2D(64, (3,3), strides=(2,2), padding='same', activation='relu'))
auto.add(MaxPooling2D((2,2), padding='same'))
auto.add(Conv2D(128, (3,3), strides=(2,2), padding='same', activation='relu'))
auto.add(MaxPooling2D((2,2), padding='same'))
auto.add(Conv2D(256, (3,3), strides=(2,2), padding='same', activation='relu'))
auto.add(MaxPooling2D((2,2), padding='same'))

# Decoder 
'''
auto.add(Conv2DTranspose(256, (2,2), strides=(2,2), activation='relu', padding='valid'))
auto.add(Conv2DTranspose(128, (2,2), strides=(2,2), activation='relu', padding='valid'))
auto.add(Conv2DTranspose(64, (2,2), strides=(2,2), activation='relu', padding='valid'))
auto.add(Conv2DTranspose(32, (2,2), strides=(2,2), activation='relu', padding='valid'))
auto.add(Conv2DTranspose(16, (2,2), strides=(2,2), activation='relu', padding='valid'))
auto.add(Conv2DTranspose(1, (2,2), strides=(2,2), activation='relu', padding='valid'))
'''
auto.add(Conv2D(256, (3,3), strides=(2,2), padding='same', activation='relu'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(128, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(64, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(32, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(16, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(8, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(4, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(2, (3,3), activation='relu', padding='same'))
auto.add(UpSampling2D((2,2)))
auto.add(Conv2D(1, (3,3), activation='sigmoid', padding='same'))

auto.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
auto.summary()
print(auto.output_shape)

In [ ]:
inputs = Input((size, size, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])
model.summary()

In [ ]:
auto = load_model(saved_model)
#auto = load_weight(saved_weights)

In [ ]:
# fit(training data, label data, ...)
steps = step_size(train_path, batch_size)

auto.fit(train_np, train_np, epochs=20, verbose=1, shuffle=True, validation_data=(validate_np, validate_np),
         steps_per_epoch=steps, validation_steps=steps)
'''
auto.fit(train_np, validate_np, validation_split=0.1, epochs=20, verbose=1, shuffle=True, 
        steps_per_epoch=epoch_steps(train_path, batch_size),
        callbacks = EarlyStopping(min_delta=0.001, patience=5))
        
auto.fit(train_np, validate_np, validation_split=0.1, epochs=epoch_size, batch_size=batch_size)
'''

In [ ]:
auto.save(os.path.join(models_path, saved_model))
#auto.save((os.path.join(models_path, saved_weights))

In [ ]:
def get_predicted(test_path):
    predicted_path = create_dir("predicted_images/")
    predicted_gray = auto.predict(validate_np)
    predicted_img = Image.fromarray(predicted_gray[0].squeeze())
    plt.imshow(predicted_img, cmap='gray')
    predicted_img.save(save_path(predicted_path, "test", "_PR"))

In [ ]:
get_predicted(validate_path)